In [2]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import binom
from statsmodels.stats.multitest import fdrcorrection 

In [4]:
# =========================================
# Circular sanity checks — distribution checks
# =========================================

TERMS = {
    "Fall 2014": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2014_ampadj.csv",
    "Spring 2015": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2015_ampadj.csv",
    "Fall 2015": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocfall2015_ampadj.csv",
    "Spring 2016": "/Users/patrycjascislewska/Library/Mobile Documents/com~apple~CloudDocs/Doktorat/Learning_management_system_USA/circular_statistics/nocspring2016_ampadj.csv",
}

MIN_EVENTS = 2


######### Circular distribution tests  #########

#Watson's test
def watson_test(theta):
    theta = np.asarray(theta, float); n = theta.size
    if n < 3:
        return np.nan, np.nan
    U = np.sort(theta % (2*np.pi)) / (2*np.pi)
    U2 = (U**2).sum() - n*U.mean()**2 - 2*(np.arange(1, n+1)*U).sum()/n + (n+1)*U.mean() + n/12 #compute Watson's U² statistic (rotation-invariant Cramér–von Mises type)
    U2 = (U2 - 0.1/n + 0.1/n**2) * (1 + 0.8/n) #small-sample correction
    p = float(np.clip(2*np.exp(-2*U2*np.pi**2), 0, 1)) #p-value approximation
    return p, float(U2)

#Rayleigh test
def rayleigh_test(alpha):
    a = np.asarray(alpha, float); n = a.size
    if n < 2:
        return np.nan, np.nan, np.nan
    C, S = np.cos(a).sum(), np.sin(a).sum() #sin and cosin for R_bar calculations, based on login times in radians
    R_bar = np.hypot(C, S) / n #Mean resultant length = R_bar, interpreted in this work as "Distinctness"
    z = n * R_bar**2 #Rayleigh test statistic
    p = np.exp(-z) * (1 + (2*z - z**2)/(4*n) - (24*z - 132*z**2 + 76*z**3 - 9*z**4)/(288*n**2)) #p-value with higher-order correction, Mardia & Jupp (2000)
    return float(R_bar), float(z), float(np.clip(p, 0, 1))

#Hodges Ajne test
def hodges_ajne_test(alpha):
    a = np.sort(np.asarray(alpha, float) % (2*np.pi)); n = a.size
    if n < 2:
        return np.nan, np.nan
    #extended angle array to handle wrap-around semicircles
    #second half is shifted by 2π so that [t, t+π) windows can cross 2π
    ext = np.concatenate([a, a + 2*np.pi])
    j, counts = 0, []
    #sliding-window algorithm: for each starting angle a[i], find how many points lie in the semicircle [a[i], a[i] + π)
    for i in range(n):
        t = a[i]
        #move j forward until ext[j] is outside the semicircle
        while j < i + n and ext[j] < t + np.pi:
            j += 1
        counts.append(j - i) #number of points in the semicircle starting at a[i]
    A = int(min(counts))
    p = 2 * min(binom.cdf(A, n, 0.5), 1 - binom.cdf(A - 1, n, 0.5))
    return float(min(p, 1.0)), A


######### Helpers #########

def round_last_to_100(pcts):
    #Round to 1 decimal place and force the last entry so the sum is 100.0
    if not pcts:
        return []
    r = [round(x, 1) for x in pcts]
    r[-1] = round(100.0 - sum(r[:-1]), 1)
    return r

#perform all tests
def process_one_term(path, term, min_events=MIN_EVENTS):
    df = pd.read_csv(path)
    df["alpha"] = df["rad"].astype(float) % (2*np.pi)

    #keep participants with at least min_events rows in the raw file
    keep = df.groupby("PSEUDO_ID").size().loc[lambda s: s >= min_events].index
    df = df[df["PSEUDO_ID"].isin(keep)]

    rows = []
    for sid, g in df.groupby("PSEUDO_ID"):
        a = (g["alpha"].dropna().to_numpy(float) % (2*np.pi))
        n = a.size
        if n < 2:
            rows.append(dict(
                TERM=term, PSEUDO_ID=sid, n_events=int(n),
                z_rayleigh=np.nan, p_rayleigh=np.nan,
                U2_watson=np.nan, p_watson=np.nan,
                A_hodges_ajne=np.nan, p_hodges_ajne=np.nan
            ))
            continue

        _, z_ray, p_ray = rayleigh_test(a)
        p_w, U2 = watson_test(a)
        p_hj, A_hj = hodges_ajne_test(a)

        rows.append(dict(
            TERM=term, PSEUDO_ID=sid, n_events=int(n),
            z_rayleigh=float(z_ray), p_rayleigh=float(p_ray),
            U2_watson=float(U2), p_watson=float(p_w),
            A_hodges_ajne=int(A_hj), p_hodges_ajne=float(p_hj)
        ))

    out = pd.DataFrame(rows)

    #FDR correction (within-term), applied over valid rows
    valid = out["n_events"] >= 2
    for col in ["p_rayleigh", "p_hodges_ajne", "p_watson"]:
        fdr_col = col + "_fdr"
        out[fdr_col] = np.nan
        mask = valid & out[col].notna()
        if mask.any():
            _, p_corr = fdrcorrection(out.loc[mask, col].to_numpy())
            out.loc[mask, fdr_col] = p_corr
    

    # Pattern labeling:
    # - If invalid (n<2): pattern = NaN
    # - Else if Hodges–Ajne (FDR) significant => non-uniform:
    #       Rayleigh (FDR) significant => unimodal
    #       Rayleigh (FDR) not significant => multimodal
    # - Else => uniform
    out.loc[~valid, "pattern"] = np.nan
    mask_nonuniform = valid & (out["p_hodges_ajne_fdr"] < 0.05)
    out.loc[mask_nonuniform & (out["p_rayleigh_fdr"] < 0.05), "pattern"] = "unimodal"
    out.loc[mask_nonuniform & ~(out["p_rayleigh_fdr"] < 0.05), "pattern"] = "multimodal"
    out.loc[valid & ~(out["p_hodges_ajne_fdr"] < 0.05), "pattern"] = "uniform"

    return out


###### RUN #######

all_results = [process_one_term(p, t) for t, p in TERMS.items()]
res_all = (
    pd.concat(all_results, ignore_index=True)
      .sort_values(["TERM", "n_events"], ascending=[True, False])
      .reset_index(drop=True)
)


#Summaries


def summarize(df, label):
    N = len(df)
    valid = df["n_events"] >= 2
    N_valid = int(valid.sum())

    def pct(count, denom):
        return "NA" if denom == 0 else f"{(100 * count / denom):.1f}%"

    #test counts -- these are uncorrected p-values, unchanged
    c_ray = int((df.loc[valid, "p_rayleigh"] < 0.05).sum())
    c_hj  = int((df.loc[valid, "p_hodges_ajne"] < 0.05).sum())
    c_w   = int((df.loc[valid, "p_watson"] < 0.05).sum())

    #pattern counts -- pattern uses FDR-adjusted logic above
    pats = df.loc[valid, "pattern"].value_counts()
    u = int(pats.get("unimodal", 0))
    m = int(pats.get("multimodal", 0))
    f = int(pats.get("uniform", 0))

    #percentages that sum to 100.0 after rounding
    pcts = [100*u/N_valid if N_valid else 0.0,
            100*m/N_valid if N_valid else 0.0,
            100*f/N_valid if N_valid else 0.0]
    pcts = round_last_to_100(pcts)

    print(f"\n=== {label} ===")
    print(f"Students: {N} (valid for tests/pattern: {N_valid}, insufficient data: {N - N_valid})")
    print(f"Rayleigh<.05: {c_ray} ({pct(c_ray, N_valid)}), "
          f"Hodges–Ajne<.05: {c_hj} ({pct(c_hj, N_valid)}), "
          f"Watson<.05: {c_w} ({pct(c_w, N_valid)})")
    print(f"Pattern → unimodal: {u} ({pcts[0]}%), "
          f"multimodal: {m} ({pcts[1]}%), "
          f"uniform: {f} ({pcts[2]}%)")

#per-term + overall summaries
for term in TERMS:
    summarize(res_all[res_all["TERM"] == term], term)
summarize(res_all, "All terms")

#peek at individual results
print("\n\nHead of individual-level results:")
print(res_all.head())


/var/folders/n4/pv0d5k_j32d9b5xf08w7ps8c0000gn/T/ipykernel_48028/3000599272.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unimodal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out.loc[mask_nonuniform & (out["p_rayleigh_fdr"] < 0.05), "pattern"] = "unimodal"
/var/folders/n4/pv0d5k_j32d9b5xf08w7ps8c0000gn/T/ipykernel_48028/3000599272.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unimodal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out.loc[mask_nonuniform & (out["p_rayleigh_fdr"] < 0.05), "pattern"] = "unimodal"
/var/folders/n4/pv0d5k_j32d9b5xf08w7ps8c0000gn/T/ipykernel_48028/3000599272.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Va


=== Fall 2014 ===
Students: 8192 (valid for tests/pattern: 8192, insufficient data: 0)
Rayleigh<.05: 6144 (75.0%), Hodges–Ajne<.05: 5652 (69.0%), Watson<.05: 6343 (77.4%)
Pattern → unimodal: 5127 (62.6%), multimodal: 235 (2.9%), uniform: 2830 (34.5%)

=== Spring 2015 ===
Students: 7711 (valid for tests/pattern: 7711, insufficient data: 0)
Rayleigh<.05: 6102 (79.1%), Hodges–Ajne<.05: 5880 (76.3%), Watson<.05: 6314 (81.9%)
Pattern → unimodal: 5500 (71.3%), multimodal: 204 (2.6%), uniform: 2007 (26.1%)

=== Fall 2015 ===
Students: 8574 (valid for tests/pattern: 8574, insufficient data: 0)
Rayleigh<.05: 6790 (79.2%), Hodges–Ajne<.05: 6526 (76.1%), Watson<.05: 6975 (81.4%)
Pattern → unimodal: 6071 (70.8%), multimodal: 249 (2.9%), uniform: 2254 (26.3%)

=== Spring 2016 ===
Students: 7841 (valid for tests/pattern: 7841, insufficient data: 0)
Rayleigh<.05: 6276 (80.0%), Hodges–Ajne<.05: 6102 (77.8%), Watson<.05: 6489 (82.8%)
Pattern → unimodal: 5691 (72.6%), multimodal: 243 (3.1%), uniform: 1

/var/folders/n4/pv0d5k_j32d9b5xf08w7ps8c0000gn/T/ipykernel_48028/3000599272.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unimodal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out.loc[mask_nonuniform & (out["p_rayleigh_fdr"] < 0.05), "pattern"] = "unimodal"
